# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 8:31AM,256881,16,9129251,Sartorius Lab Products and Service,Released
1,Feb 8 2023 8:31AM,256881,16,9129252,Sartorius Lab Products and Service,Released
2,Feb 8 2023 8:31AM,256881,16,9129269,Sartorius Lab Products and Service,Released
3,Feb 8 2023 8:31AM,256880,16,9128696,Sartorius Bioprocess Solutions,Released
4,Feb 8 2023 8:30AM,256865,10,0086362175,ISDIN Corporation,Released
5,Feb 8 2023 8:30AM,256857,10,PRF-52211,Bio-PRF,Released
6,Feb 8 2023 8:30AM,256857,10,PRF-52215,Bio-PRF,Released
7,Feb 8 2023 8:30AM,256857,10,PRF-52219,Bio-PRF,Released
8,Feb 8 2023 8:30AM,256857,10,PRF-52225,Bio-PRF,Released
9,Feb 8 2023 8:30AM,256857,10,PRF-52228,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,256877,Released,6
41,256878,Released,3
42,256879,Released,1
43,256880,Released,1
44,256881,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256877,NaN,NaN,6.0
256878,NaN,NaN,3.0
256879,NaN,NaN,1.0
256880,NaN,NaN,1.0
256881,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256782,84.0,0.0,21.0
256783,25.0,0.0,27.0
256798,0.0,0.0,1.0
256805,0.0,0.0,1.0
256807,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256782,84,0,21
256783,25,0,27
256798,0,0,1
256805,0,0,1
256807,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256782,84,0,21
1,256783,25,0,27
2,256798,0,0,1
3,256805,0,0,1
4,256807,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256782,84,,21
1,256783,25,,27
2,256798,,,1
3,256805,,,1
4,256807,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 8:31AM,256881,16,Sartorius Lab Products and Service
3,Feb 8 2023 8:31AM,256880,16,Sartorius Bioprocess Solutions
4,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation
5,Feb 8 2023 8:30AM,256857,10,Bio-PRF
17,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation
22,Feb 8 2023 8:30AM,256856,10,ISDIN Corporation
34,Feb 8 2023 8:20AM,256879,16,"Emersa Waterbox, LLC"
35,Feb 8 2023 8:10AM,256878,19,"AdvaGen Pharma, Ltd"
38,Feb 8 2023 8:06AM,256877,19,"AdvaGen Pharma, Ltd"
44,Feb 7 2023 4:33PM,256783,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 8:31AM,256881,16,Sartorius Lab Products and Service,,,3
1,Feb 8 2023 8:31AM,256880,16,Sartorius Bioprocess Solutions,,,1
2,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation,,,14
3,Feb 8 2023 8:30AM,256857,10,Bio-PRF,,,11
4,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation,,,1
5,Feb 8 2023 8:30AM,256856,10,ISDIN Corporation,,,4
6,Feb 8 2023 8:20AM,256879,16,"Emersa Waterbox, LLC",,,1
7,Feb 8 2023 8:10AM,256878,19,"AdvaGen Pharma, Ltd",,,3
8,Feb 8 2023 8:06AM,256877,19,"AdvaGen Pharma, Ltd",,,6
9,Feb 7 2023 4:33PM,256783,15,"Brookfield Pharmaceuticals, LLC",25,,27


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:31AM,256881,16,Sartorius Lab Products and Service,3,,
1,Feb 8 2023 8:31AM,256880,16,Sartorius Bioprocess Solutions,1,,
2,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation,14,,
3,Feb 8 2023 8:30AM,256857,10,Bio-PRF,11,,
4,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation,1,,
5,Feb 8 2023 8:30AM,256856,10,ISDIN Corporation,4,,
6,Feb 8 2023 8:20AM,256879,16,"Emersa Waterbox, LLC",1,,
7,Feb 8 2023 8:10AM,256878,19,"AdvaGen Pharma, Ltd",3,,
8,Feb 8 2023 8:06AM,256877,19,"AdvaGen Pharma, Ltd",6,,
9,Feb 7 2023 4:33PM,256783,15,"Brookfield Pharmaceuticals, LLC",27,,25


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:31AM,256881,16,Sartorius Lab Products and Service,3,,
1,Feb 8 2023 8:31AM,256880,16,Sartorius Bioprocess Solutions,1,,
2,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation,14,,
3,Feb 8 2023 8:30AM,256857,10,Bio-PRF,11,,
4,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:31AM,256881,16,Sartorius Lab Products and Service,3.0,NaN,NaN
1,Feb 8 2023 8:31AM,256880,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation,14.0,NaN,NaN
3,Feb 8 2023 8:30AM,256857,10,Bio-PRF,11.0,NaN,NaN
4,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:31AM,256881,16,Sartorius Lab Products and Service,3.0,0.0,0.0
1,Feb 8 2023 8:31AM,256880,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation,14.0,0.0,0.0
3,Feb 8 2023 8:30AM,256857,10,Bio-PRF,11.0,0.0,0.0
4,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3852565,91.0,9.0,2.0
12,256841,0.0,1.0,3.0
15,513565,48.0,0.0,109.0
16,1541152,7.0,1.0,0.0
19,1541133,35.0,12.0,0.0
20,256825,0.0,1.0,0.0
21,1284170,5.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3852565,91.0,9.0,2.0
1,12,256841,0.0,1.0,3.0
2,15,513565,48.0,0.0,109.0
3,16,1541152,7.0,1.0,0.0
4,19,1541133,35.0,12.0,0.0
5,20,256825,0.0,1.0,0.0
6,21,1284170,5.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,91.0,9.0,2.0
1,12,0.0,1.0,3.0
2,15,48.0,0.0,109.0
3,16,7.0,1.0,0.0
4,19,35.0,12.0,0.0
5,20,0.0,1.0,0.0
6,21,5.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,91.0
1,12,Released,0.0
2,15,Released,48.0
3,16,Released,7.0
4,19,Released,35.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,2.0,3.0,109.0,0.0,0.0,0.0,0.0
Executing,9.0,1.0,0.0,1.0,12.0,1.0,0.0
Released,91.0,0.0,48.0,7.0,35.0,0.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,2.0,3.0,109.0,0.0,0.0,0.0,0.0
1,Executing,9.0,1.0,0.0,1.0,12.0,1.0,0.0
2,Released,91.0,0.0,48.0,7.0,35.0,0.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,2.0,3.0,109.0,0.0,0.0,0.0,0.0
1,Executing,9.0,1.0,0.0,1.0,12.0,1.0,0.0
2,Released,91.0,0.0,48.0,7.0,35.0,0.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()